# WASHINGTON DC CRIME ANALYSIS

FROM KAGGLE:
    
Dataset of all of the crimes in the DC metro police system ranging from Theft, Arson, Assault, Homicide, Sex Abuse, Robbery, and Burglary.

Data can be easily geocoded and mapped, trends can be extracted, and predictions can be made.

Would be interesting to combine with other datasets, i.e. changes in housing prices, history of construction sites etc. 
j
An informal hypothesis would be: If the local government invests in fixing the sidewalks in a neighborhood, how much would the investment decrease crime levels on a block by block basis.

Raw Data can be accessed from:
http://crimemap.dc.gov/CrimeMapSearch.aspx#tabs-GeoOther

The data is most easily accessed by downloading 1 ward at a time for the specific data range.

## PHASE 1: ENVIRONMENT SETUP

In [2]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import csv
from scipy import stats
import seaborn as sns; sns.set() # use seaborn plotting defaults
from sklearn.svm import SVC  

## PHASE 2: ETL

In [5]:
dc_raw = df = pd.read_csv('data/dc_crime_add_vars.csv', index_col=[0])
dc_raw

,X,REPORT_DAT,SHIFT,OFFENSE,METHOD,BLOCK,DISTRICT,PSA,WARD,ANC,...,year,month,day,hour,minute,second,EW,NS,quad,crimetype
1,1,8/31/2008 8:47:00 PM,EVENING,THEFT/OTHER,OTHERS,3500 - 3599 BLOCK OF R STREET NW,2.0,206.0,2,2E,...,2008,8,31,20,47,0,West,North,Northwest,Non-Violent
2,2,9/1/2008 12:45:00 AM,MIDNIGHT,MOTOR VEHICLE THEFT,OTHERS,2000 - 2015 BLOCK OF O STREET NW,2.0,208.0,2,2B,...,2008,9,1,0,45,0,West,North,Northwest,Non-Violent
3,3,9/1/2008 3:00:00 AM,MIDNIGHT,THEFT/OTHER,OTHERS,700 - 799 BLOCK OF 11TH STREET NW,1.0,101.0,2,2C,...,2008,9,1,3,0,0,East,North,Northeast,Non-Violent
4,4,9/9/2008 7:46:00 AM,DAY,THEFT/OTHER,OTHERS,1700 - 1799 BLOCK OF P STREET NW,2.0,208.0,2,2B,...,2008,9,9,7,46,0,West,North,Northwest,Non-Violent
5,5,8/24/2008 8:00:00 PM,EVENING,MOTOR VEHICLE THEFT,OTHERS,500 - 599 BLOCK OF INDIANA AVENUE NW,1.0,102.0,2,2C,...,2008,8,24,20,0,0,East,North,Northeast,Non-Violent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342863,342863,3/21/2008 1:00:00 PM,DAY,MOTOR VEHICLE THEFT,OTHERS,2200 - 2230 BLOCK OF 6TH STREET NW,3.0,305.0,1,1B,...,2008,3,21,13,0,0,East,North,Northeast,Non-Violent
342864,342864,3/21/2008 9:08:00 PM,EVENING,THEFT/OTHER,OTHERS,3300 - 3341 BLOCK OF 16TH STREET NW,3.0,302.0,1,1D,...,2008,3,21,21,8,0,West,North,Northwest,Non-Violent
342865,342865,3/21/2008 4:30:00 PM,EVENING,THEFT/OTHER,OTHERS,1500 - 1530 BLOCK OF PARK ROAD NW,3.0,302.0,1,1A,...,2008,3,21,16,30,0,East,North,Northeast,Non-Violent
342866,342866,3/21/2008 5:15:00 PM,EVENING,MOTOR VEHICLE THEFT,OTHERS,3200 - 3299 BLOCK OF PARK PLACE NW,3.0,302.0,1,1A,...,2008,3,21,17,15,0,East,North,Northeast,Non-Violent


In [9]:

# retrieve administrative information of the dataframe 
df_size = dc_raw.size
df_size

10628877

In [10]:
dc_raw.shape


(342867, 31)

In [11]:
dc_raw.ndim

2

In [13]:

# create a list of all the column headers
col_list = dc_raw.columns.tolist()
col_list

['X',
 'REPORT_DAT',
 'SHIFT',
 'OFFENSE',
 'METHOD',
 'BLOCK',
 'DISTRICT',
 'PSA',
 'WARD',
 'ANC',
 'NEIGHBORHOOD_CLUSTER',
 'BLOCK_GROUP',
 'CENSUS_TRACT',
 'VOTING_PRECINCT',
 'CCN',
 'START_DATE',
 'END_DATE',
 'XBLOCK',
 'YBLOCK',
 'optional',
 'date',
 'year',
 'month',
 'day',
 'hour',
 'minute',
 'second',
 'EW',
 'NS',
 'quad',
 'crimetype']

In [15]:
dc_raw.SHIFT.unique()

array(['EVENING', 'MIDNIGHT', 'DAY'], dtype=object)

# PHASE 3: DATA EXPLORATION